In [1]:
import pandas as pd
import numpy as np
import xlrd
import sys
#import Adafruit_DHT
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from termcolor import colored
import urllib.request, urllib.error, urllib.parse,json
  
a=int(100000)
num_samples=a
#sensor = Adafruit_DHT.DHT22
#pin = 14
READ_API_KEY='9CXXBH0CC8HP4NSA'
CHANNEL_ID=974692
CHANNEL_ID1=1014134
READ_API_KEY1='CB3Y5ICWLYT0JJD0'

def main():
    conn = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                           % (CHANNEL_ID,READ_API_KEY))

    response = conn.read()
    #print ("http status code=%s" % (conn.getcode()))
    data=json.loads(response)
    #print (data['field1'],data['field2'],data['created_at'])
    conn.close()
    Humidity=float(data['field1'])
    Temp=float(data['field2'])
    humi = Humidity                     
    temp = Temp
    #print("Temperature={}, Humidity={}".format(temp,humi))
    return temp,humi
def main1():
    conn = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                           % (CHANNEL_ID1,READ_API_KEY1))

    response = conn.read()
    #print ("http status code=%s" % (conn.getcode()))
    data=json.loads(response)
    #print (data['field1'],data['field2'],data['created_at'])
    conn.close()
    pH=float(data['field1'])
    #print("Temperature={}, Humidity={}".format(temp,humi))
    return pH



def get_input_and_output(num_samples=a):
    # Read excel file
    column_names = ["Crop", "Season",
                    "Rainfall Min",
                    "Rainfall Max",
                    "Relative Humidity Min",
                    "Relative Humidity Max",
                    "Min Temp", "Max Temp",
                    "pH Min", "pH Max", ]
    loc = (r"/content/drive/My Drive/Crop.xlsx")
    df = pd.read_excel(loc, names=column_names)

    # Generate crop ids
    unique_crops = df.Crop.unique()
    crop_ids = {crop: i for i, crop in enumerate(unique_crops)}

    # Generate season ids
    unique_seasons = df.Season.unique()
    season_ids = {season: i for i, season in enumerate(unique_seasons)}

    # Assign ids in data
    # df.replace({'Crop': crop_ids, 'Season': season_ids}, inplace=True)

    data_values = []

    input_len = len(df)
    for i in range(input_len):
        crop = df.loc[i, "Crop"]
        season = df.loc[i, "Season"]
        rainfall_min = float(df.loc[i, "Rainfall Min"])
        rainfall_max = float(df.loc[i, "Rainfall Max"])
        relative_humidity_min = float(df.loc[i, "Relative Humidity Min"])
        relative_humidity_max = float(df.loc[i, "Relative Humidity Max"])
        min_temp = float(df.loc[i, "Min Temp"])
        max_temp = float(df.loc[i, "Max Temp"])
        p_h_min = float(df.loc[i, "pH Min"])
        p_h_max = float(df.loc[i, "pH Max"])

        for j in range(num_samples):
            obj = [
                crop_ids[crop],
                season_ids[season],
                random.uniform(rainfall_min, rainfall_max),
                random.uniform(relative_humidity_min, relative_humidity_max),
                random.uniform(min_temp, max_temp),
                random.uniform(p_h_min, p_h_max),
            ]
            data_values.append(obj)

    final_cols = ["Crop", "Season", "Rainfall", "Humidity", "Temp", "pH"]
    final_df = pd.DataFrame(data_values, columns=final_cols)

    # Separate input and output as X and y respectively
    X = final_df.iloc[:, 1:].values
    y = final_df.iloc[:, 0].values
    return X, y, crop_ids, season_ids




def get_trained_model(num_samples):
    X, y, crop_ids, season_ids = get_input_and_output(num_samples)
    knn_classifier = KNeighborsClassifier(n_neighbors=3)
    # Train the KNN Model
    knn_classifier.fit(X, y)
    return knn_classifier, crop_ids, season_ids



def test_model():
    #print("Evaulating model with {} samples from ranges of each crop".format(a))
    X, y, _, _ = get_input_and_output()
    # Split the data for training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    # Create KNN Model object with k=3
    knn_classifier = KNeighborsClassifier(n_neighbors=3)
    # Train the KNN Model
    knn_classifier.fit(X_train, y_train)

    # predict the test values
    y_pred = knn_classifier.predict(X_test)

    # Evaluate model
    score = accuracy_score(y_test, y_pred)
    print("accuracy:\t%0.3f" % score)
    confusion_matrix_val = confusion_matrix(y_test, y_pred)
    #print(confusion_matrix_val)
    print(classification_report(y_test, y_pred,))
    print(score)

def predict(num_samples):
    season = input('Enter the Season: ')
    rf = float(input('Enter the Rainfall of the area: '))
    rh = humi
    Temp = temp
    ph = pH

    model, crop_ids, season_ids = get_trained_model(num_samples)

    X_test = np.array([[season_ids[season], rf, rh, Temp, ph]])
    # X_test = np.array([[season_ids['Rabi'], 800.0, 80.0, 13.0, 6.0]])
    y_pred = model.predict(X_test)
    reverse_crop_ids = {val: key for key, val in crop_ids.items()}
    cropname=reverse_crop_ids[int(y_pred[0])]
    cropname1 = int(y_pred[0])
    
    return reverse_crop_ids,cropname,y_pred,X_test,cropname1


loc1 = (r"/content/drive/My Drive/Book1.xlsx")
df1=pd.read_excel(loc1)
def fertilizer(loc1):
    wb = xlrd.open_workbook(loc1) 
    sheet = wb.sheet_by_index(0)
    #print(wb)
    fertilizerrow=sheet.row_values(int(e))
    #print( fertilizerrow)
    return wb,sheet,fertilizerrow


def table(sheet,e):
    for i in range(sheet.ncols): 
        greenc=colored('{}', 'green').format(sheet.cell_value(0, i))
        redc=colored('{}', 'red').format(sheet.cell_value(e+1, i))
        print("{}:{}".format(greenc,redc))


if __name__ == '__main__':
    temp,humi=main()
    pH=main1()
    print("Temperature = {}, Humidity = {}, pH = {}".format(temp,humi,pH))
    test_model()
    a,b,c,d,e = predict(num_samples)
    print("The id is {}".format(e))
    print("The predicted crop is {}".format(b))
    wb,sheet,fertilizerrow=fertilizer(loc1)
    table(sheet,e)






Temperature = 31.8, Humidity = 62.9, pH = 4.05
accuracy:	0.871
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     20065
           1       0.83      0.88      0.86     19872
           2       0.93      0.97      0.95     19962
           3       0.86      0.89      0.87     19956
           4       0.83      0.91      0.87     19905
           5       1.00      1.00      1.00     20101
           6       1.00      1.00      1.00     19885
           7       0.98      1.00      0.99     20129
           8       0.84      0.89      0.86     20059
           9       0.91      0.99      0.95     20173
          10       0.94      0.99      0.96     19902
          11       1.00      1.00      1.00     20188
          12       0.68      0.82      0.75     20053
          13       0.85      0.72      0.78     19910
          14       0.95      0.99      0.97     20187
          15       0.93      0.97      0.95     19811
          16      

KeyboardInterrupt: ignored